In [1]:
import pandas as pd
import numpy as np
import time
from keras.models import Sequential
from keras.layers import Dense,RepeatVector,Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

In [2]:
data = pd.read_csv('AMD.csv')

In [3]:
data

,ticker,date,open,high,low,close
0,AMD,1980-03-17,0.00,3.30,3.12,3.15
1,AMD,1980-03-18,0.00,3.12,2.94,3.03
2,AMD,1980-03-19,0.00,3.08,3.02,3.04
3,AMD,1980-03-20,0.00,3.06,3.01,3.01
4,AMD,1980-03-21,0.00,3.02,2.91,2.92
...,...,...,...,...,...,...
10860,AMD,2023-04-13,92.79,93.16,91.83,92.09
10861,AMD,2023-04-14,91.82,92.97,90.50,91.75
10862,AMD,2023-04-17,90.23,90.69,88.30,89.87
10863,AMD,2023-04-18,91.61,92.16,89.33,89.78


In [4]:
data = data.filter(['close'])

In [5]:
# Lấy 100 dòng cuối cùng của df
data = data.tail(1000)
data

,close
9865,26.81
9866,28.29
9867,28.22
9868,27.42
9869,26.66
...,...
10860,92.09
10861,91.75
10862,89.87
10863,89.78


In [6]:
#Tiền xử lý
data.fillna(0, inplace=True)

# Normalize data
scaler = MinMaxScaler()
dataScale = scaler.fit_transform(data.values.reshape(-1,1))

In [7]:
#Chia dữ liệu thành train test
data = pd.DataFrame(dataScale)

#dataScale = data
train_percent = 0.7
train_size = int(len(data) * train_percent)
train_data = data.iloc[:train_size]
test_data = data.iloc[train_size:]

In [8]:
#Chuẩn bị dữ liệu để đưa vào mô hình
def prepare_data(data, size_window,size_predict,stepWindow):
    X, y = [], []
    startWindow = 0
    for i in range(len(data) - size_window - 1):
        if (len(data[(startWindow + size_window):(startWindow + size_window + size_predict) , 0]) != size_predict):
            break
        X.append(data[startWindow:(startWindow + size_window), :])
        y.append(data[(startWindow + size_window):(startWindow + size_window + size_predict) , 0])
          
       
        startWindow += stepWindow
    return np.array(X), np.array(y)


size_window = 7
size_predict = 1
stepWindow = 1
X_train, y_train = prepare_data(train_data.values, size_window, size_predict, stepWindow)
X_test, y_test = prepare_data(test_data.values, size_window, size_predict, stepWindow)

In [9]:
start_time = time.time()

In [10]:
lucas_list =[3, 4, 7, 11, 18, 29, 47, 76]
lucas_number = lucas_list[1]

In [11]:

# Tạo mô hình CNN
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(size_window, 1)))

model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
for i in range(lucas_number):
    model.add(Dense(units=100, activation='relu')) 
    model.add(Dropout(0.2))
    model.add(Dense(units=1))

# Compile mô hình với Adam optimizer và Mean Squared Error loss
model.compile(loss='mse', optimizer='adam')


In [12]:
history = model.fit(X_train, y_train, epochs=11000, batch_size=64)

Epoch 1/11000
11/11 [==============================] - 2s 3ms/step - loss: 0.1486
Epoch 2/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0897
Epoch 3/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0597
Epoch 4/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0536
Epoch 5/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0325
Epoch 6/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0134
Epoch 7/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0094
Epoch 8/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0074
Epoch 9/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0069
Epoch 10/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0067
Epoch 11/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0062
Epoch 12/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0058
Epoch 13/1100

11/11 [==============================] - 0s 2ms/step - loss: 0.0019
Epoch 197/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0019
Epoch 198/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0019
Epoch 199/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0019
Epoch 200/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0017
Epoch 201/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0016
Epoch 202/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0019
Epoch 203/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0017
Epoch 204/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0017
Epoch 205/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0018
Epoch 206/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0018
Epoch 207/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0014
Epoch 20

11/11 [==============================] - 0s 2ms/step - loss: 0.0017
Epoch 391/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0015
Epoch 392/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0014
Epoch 393/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0015
Epoch 394/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0017
Epoch 395/11000
11/11 [==============================] - 0s 3ms/step - loss: 0.0015
Epoch 396/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0016
Epoch 397/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0016
Epoch 398/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0015
Epoch 399/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0016
Epoch 400/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0015
Epoch 401/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0014
Epoch 40

11/11 [==============================] - 0s 2ms/step - loss: 0.0015
Epoch 585/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0016
Epoch 586/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0016
Epoch 587/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0014
Epoch 588/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0015
Epoch 589/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0016
Epoch 590/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0016
Epoch 591/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0013
Epoch 592/11000
11/11 [==============================] - 0s 3ms/step - loss: 0.0015
Epoch 593/11000
11/11 [==============================] - 0s 3ms/step - loss: 0.0015
Epoch 594/11000
11/11 [==============================] - 0s 3ms/step - loss: 0.0015
Epoch 595/11000
11/11 [==============================] - 0s 3ms/step - loss: 0.0015
Epoch 59

11/11 [==============================] - 0s 2ms/step - loss: 0.0014
Epoch 779/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0014
Epoch 780/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0015
Epoch 781/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0018
Epoch 782/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0014
Epoch 783/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0016
Epoch 784/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0015
Epoch 785/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0017
Epoch 786/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0014
Epoch 787/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0014
Epoch 788/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0014
Epoch 789/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0014
Epoch 79

...

11/11 [==============================] - 0s 2ms/step - loss: 0.0013
Epoch 10187/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0011
Epoch 10188/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0012
Epoch 10189/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0012
Epoch 10190/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0011
Epoch 10191/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0012
Epoch 10192/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0011
Epoch 10193/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0011
Epoch 10194/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0012
Epoch 10195/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0011
Epoch 10196/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0012
Epoch 10197/11000
11/11 [==============================] - 0s 2ms/step -

11/11 [==============================] - 0s 2ms/step - loss: 0.0010
Epoch 10377/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0010
Epoch 10378/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0011
Epoch 10379/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0011
Epoch 10380/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0013
Epoch 10381/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0012
Epoch 10382/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0014
Epoch 10383/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0013
Epoch 10384/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0011
Epoch 10385/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0012
Epoch 10386/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0011
Epoch 10387/11000
11/11 [==============================] - 0s 2ms/step -

11/11 [==============================] - 0s 2ms/step - loss: 0.0011
Epoch 10567/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0011
Epoch 10568/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0013
Epoch 10569/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0012
Epoch 10570/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0012
Epoch 10571/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0013
Epoch 10572/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0011
Epoch 10573/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0011
Epoch 10574/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0010
Epoch 10575/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0013
Epoch 10576/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0012
Epoch 10577/11000
11/11 [==============================] - 0s 2ms/step -

11/11 [==============================] - 0s 2ms/step - loss: 0.0011
Epoch 10757/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0010
Epoch 10758/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0012
Epoch 10759/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0014
Epoch 10760/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0012
Epoch 10761/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0011
Epoch 10762/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0012
Epoch 10763/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0012
Epoch 10764/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0012
Epoch 10765/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0012
Epoch 10766/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0013
Epoch 10767/11000
11/11 [==============================] - 0s 2ms/step -

11/11 [==============================] - 0s 2ms/step - loss: 0.0013
Epoch 10947/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0011
Epoch 10948/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0012
Epoch 10949/11000
11/11 [==============================] - 0s 2ms/step - loss: 9.5103e-04
Epoch 10950/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0012
Epoch 10951/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0010
Epoch 10952/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0013
Epoch 10953/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0012
Epoch 10954/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0013
Epoch 10955/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0011
Epoch 10956/11000
11/11 [==============================] - 0s 2ms/step - loss: 0.0010
Epoch 10957/11000
11/11 [==============================] - 0s 2ms/st

In [14]:
y_pred = model.predict(X_test)
end_time = time.time()
training_time = end_time - start_time
print(f"Training time: {training_time} seconds")

10/10 [==============================] - 0s 1ms/step
Training time: 438.20401525497437 seconds


In [15]:
import h5py
from keras.models import save_model

# Lưu mô hình CNN vào file model.h5
save_model(model, 'CNN_AMD_73.h5')

# Tạo file H5 mới
with h5py.File('CNN_AMD_73_data.h5', 'w') as f:
    # Lưu giá trị y_pred
    f.create_dataset('y_test', data=y_test)
    
    # Lưu giá trị X_test
    f.create_dataset('X_test', data=X_test)
    
    # Lưu giá trị training_time
    f.create_dataset('training_time', data=training_time)
